In [ ]:
!conda install -y -c conda-forge mongodb

Go to teh terminal or command prompt and start the mongodb
    
    "~/anaconda3/bin/mongod --dbpath ~Desktop/10_database/02_mongodb"

    "C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe" --dbpath d:\test\mongodb\data

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pymongo

In [ ]:
import pymongo

In [ ]:
client=pymongo.MongoClient('localhost',27017)

In [ ]:
db=client['chapter3']

In [ ]:
collection=db['income']

In [ ]:
with open('income_header.txt') as f_in:
    column_headings=f_in.readline()

In [ ]:
column_headings

In [ ]:
column_headings_list=column_headings.split(',')

In [ ]:
column_headings_list

In [ ]:
#cleaning 

row_dict_list = list()

with open('income.txt') as f_in:
    for line in  f_in:
        row_list=line.rstrip('\n').split(',')
        row_dict=dict(zip(column_headings_list,row_list))
        row_dict_list.append(row_dict)
        collection.insert_one(row_dict)  # put it in database

In [ ]:
collection.estimated_document_count()

In [ ]:
# perform quality checks

age39=collection.find_one({'age':39})

In [ ]:
collection.delete_many({})

In [ ]:
collection.count()

In [ ]:
#converting age to integer
with open('income.txt') as f_in:
    for line in  f_in:
        row_list=line.rstrip('\n').split(',')
        row_dict=dict(zip(column_headings_list,row_list))
        try:
            row_dict['age']=int(row_dict['age'])
            row_dict_list.append(row_dict)
            collection.insert_one(row_dict)#put it in database
        except:
            pass

In [ ]:
over_35=collection.find({'age':{"$gt":35}})

In [ ]:
type(over_35)

In [ ]:
over_35.next()

In [ ]:
over_35.count()

In [ ]:
import time
start=time.time()
#age50=collection.find({'age':{'$eq':50}})
over_35=collection.find({'age':{"$gt":55}})
end=time.time()
print(end - start)

In [ ]:
#creating index
index_result=db.profiles.create_index([('age',pymongo.ASCENDING)],unique=False)

In [ ]:
import time
start=time.time()
#age50=collection.find({'age':{'$eq':50}})
over_35=collection.find({'age':{"$gt":55}})
end=time.time()
print(end - start)

In [ ]:

###Analytics with Cassendra
from cassandra.cluster import Cluster
cluster=Cluster(['localhost'])# expects a list as parameter

In [ ]:
session=cluster.connect()

In [ ]:
session.execute("CREATE KEYSPACE stocks\
                    WITH replication={'class':'SimpleStrategy',\
                                        'replication_factor':'1'}")#run the query

In [ ]:
session.set_keyspace('stocks')

In [ ]:
session.execute("""
                    CREATE TABLE company(
                        company_id text,
                        name_latest text,
                        name_previous text,
                        PRIMARY KEY(company_id))""")

In [ ]:
session.execute("""
                    CREATE TABLE indicator_by_company(
                        company_id text,
                        indicator_id text,
                        yr_2010 bigint,
                        yr_2011 bigint,
                        yr_2012 bigint,
                        yr_2013 bigint,
                        yr_2014 bigint,
                        yr_2015 bigint,
                        yr_2016 bigint,
                        PRIMARY KEY(company_id,indicator_id))""")

In [ ]:
#load data into table
import json
with open("companies.json") as f_in:
    companies=json.load(f_in)
    

In [ ]:
type(companies)

In [ ]:
companies[0]

In [ ]:
session.execute("""
                    INSERT INTO company (company_id,name_latest,name_previous)
                    VALUES (%s,%s,%s)
                    """,
                    ("1000045", "Nicholas Financial Inc",""))

In [ ]:
result_set=session.execute("SELECT * FROM company")

In [ ]:
result_set.current_rows

In [ ]:
for company in companies:
    try:
        session.execute("""
                        
                        INSERT INTO company (company_id,name_latest,name_previous)
                        VALUES (%s,%s,%s)
                        """,
                        (company['company_id'],company['name_latest'],company['names_previous']))
    except:
        pass

In [ ]:
result_set=session.execute("SELECT * FROM company")

In [ ]:
result_set

In [ ]:
import pandas as pd
income_df=pd.DataFrame(list(collection.find()))
income_df.head()
income_df['age'].describe()
income_df.mean(1)